In [ ]:
!pip install -qU cohere  gradio


In [ ]:
# Install eSpeak-ng
!apt-get install -y espeak-ng

In [ ]:
!pip install gtts

In [ ]:
def clean_script(script):
    # Remove stage directions and unwanted lines
    script = re.sub(r"\*\*?\[.*?\]\*\*?", "", script)              # Removes [Intro Music Fades In], [Music Transition], etc.
    script = re.sub(r"\bHost:?\b", "", script, flags=re.IGNORECASE) # Removes 'Host:' or 'host:'
    script = re.sub(r"I[’']m your host.*?,", "", script, flags=re.IGNORECASE)  # Removes lines like "I’m your host, [Your Name],"
    script = re.sub(r"\[Your Name\]", "", script, flags=re.IGNORECASE)         # Removes placeholder names
    script = re.sub(r"\s{2,}", " ", script)                        # Remove extra spaces
    script = re.sub(r"\n{2,}", "\n", script)                       # Remove extra newlines
    script = re.sub(r"\*\*Podcast Script.*?\*\*", "", script)    # Remove '**Podcast Script...**'
    script = re.sub(r"I[’']m\s+,\s+and\s+today\s+on\s+\[Podcast\s+Name\]", "", script) # Remove '! I’m , and today on [Podcast Name]'
    script = re.sub(r"\*\*?\s*:\s*\*\*?", "", script)             # Remove '**:**' and similar
    script = re.sub(r"End of Script\.", "", script)               # Remove 'End of Script.'
    script = re.sub(r"I[’']m\s+,\s+and\s+this\s+has\s+been", "", script) # Remove "I’m , and this has been"

    return script.strip()

In [ ]:
!pip install cohere gtts

import gradio as gr
import cohere
from gtts import gTTS
import os
import uuid
import re

# 1. Initialize Cohere API
co = cohere.Client("GARNtcnzo4a0k1xazRIZf1SBUUqbFrSn9uIX7B39")  # 🔑 Replace with your real key

# 2. Generate podcast script
def generate_script(topic):
    prompt = f"Write a 400-word podcast script on the topic: '{topic}'."
    # Using Cohere's chat endpoint
    response = co.chat(
        model="command-a-translate-08-2025",
        message=prompt,
        max_tokens=800,
        temperature=0.7
    )

    # Handle possible response formats
    if hasattr(response, "text") and response.text:
        script = response.text.strip()
    elif hasattr(response, "output_text"):
        script = response.output_text.strip()
    else:
        raise RuntimeError("⚠️ Cohere response format not recognized.")

    # Clean the script
    return clean_script(script)

# 3. Function to clean the script
def clean_script(script):
    # Remove stage directions and unwanted lines
    script = re.sub(r"\*\*?\[.*?\]\*\*?", "", script)              # Removes [Intro Music Fades In], [Music Transition], etc.
    script = re.sub(r"\bHost:?\b", "", script, flags=re.IGNORECASE) # Removes 'Host:' or 'host:'
    script = re.sub(r"I[’']m your host.*?,", "", script, flags=re.IGNORECASE)  # Removes lines like "I’m your host, [Your Name],"
    script = re.sub(r"\[Your Name\]", "", script, flags=re.IGNORECASE)         # Removes placeholder names
    script = re.sub(r"\s{2,}", " ", script)                        # Remove extra spaces
    script = re.sub(r"\n{2,}", "\n", script)                       # Remove extra newlines
    script = re.sub(r"\*\*Podcast Script.*?\*\*", "", script)    # Remove '**Podcast Script...**'
    script = re.sub(r"I[’']m\s+,\s+and\s+today\s+on\s+\[Podcast\s+Name\]", "", script) # Remove '! I’m , and today on [Podcast Name]'
    script = re.sub(r"\*\*?\s*:\s*\*\*?", "", script)             # Remove '**:**' and similar
    script = re.sub(r"End of Script\.", "", script)               # Remove 'End of Script.'
    script = re.sub(r"I[’']m\s+,\s+and\s+this\s+has\s+been", "", script) # Remove "I’m , and this has been"
    script = re.sub(r"\*\*Segment \d+:\s*.*?\*\*", "", script)     # Remove **Segment X: ...**

    return script.strip()


# 4. Convert cleaned script to speech with gTTS
def generate_audio(script):
    filename = f"podcast_{uuid.uuid4().hex}.mp3"
    tts = gTTS(text=script, lang="en")
    tts.save(filename)
    return filename

# 5. Full workflow
def generate_podcast(topic):
    print(f"🎙️ Generating podcast on: {topic}")

    script = generate_script(topic)
    print("\n📜 Cleaned Script Generated:\n")
    print(script)

    print("\n🔊 Generating audio...")
    audio_file = generate_audio(script)
    print(f"✅ Podcast saved as: {audio_file}")

    return script, audio_file


# Gradio Interface
iface = gr.Interface(
    fn=generate_podcast,
    inputs="text",
    outputs=["text", "audio"],
    title="Podcast Generator",
    description="Enter a topic and generate a podcast script and audio."
)

iface.launch(debug=True, share=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 15.5 MB/s eta 0:00:00
  Attempting uninstall: httpx-sse
    Found existing installation: httpx-sse 0.4.3
    Uninstalling httpx-sse-0.4.3:
      Successfully uninstalled httpx-sse-0.4.3
  Attempting uninstall: click
    Found existing installation: click 8.3.0
    Uninstalling click-8.3.0:
      Successfully uninstalled click-8.3.0
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e5701e2d3f60cec441.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🎙️ Generating podcast on: ben 10

📜 Cleaned Script Generated:

**Intro Music Fades In**  "Welcome back to *Nostalgia Nexus*, the podcast where we dive into the shows, movies, and characters that shaped our childhoods. I’m your , , and today we’re stepping into the world of one of the most iconic animated series of the 2000s—*Ben 10*. Whether you were a kid who dreamed of having the Omnitrix or an adult who secretly wished you could transform into an alien, this show left an indelible mark on pop culture. So, let’s strap in and explore the legacy of Ben 10." **Segment 1: The Origins of Ben 10** "Ben 10 first premiered on Cartoon Network in 2005, created by Man of Action—a group of writers and artists who also worked on *Generator Rex* and *Big Hero 6*. The story follows Ben Tennyson, a 10-year-old boy who discovers a mysterious watch-like device called the Omnitrix during a summer vacation with his cousin Gwen and grandfather Max. The Omnitrix allows Ben to transform into ten different 